In [1]:
import subprocess
import json

import pandas as pd

from helpers import *

In [2]:
TARGET_COLUMNS = [
    "hospital_id",
    #'row_id',
    "line_type",
    "description",
    "rev_code",
    "local_code",
    "code",
    "ms_drg",
    "apr_drg",
    "eapg",
    "hcpcs_cpt",
    "modifiers",
    "alt_hcpcs_cpt",
    "thru",
    "apc",
    "icd",
    "ndc",
    "drug_hcpcs_multiplier",
    "drug_quantity",
    "drug_unit_of_measurement",
    "drug_type_of_measurement",
    "billing_class",
    "setting",
    "rate_category",
    "payer_name",
    "plan_name",
    "standard_charge",
    "standard_charge_percent",
    "contracting_method",
    "additional_generic_notes",
    "additional_payer_specific_notes",
]

In [3]:
TRANSPARENCY_PAGE = "https://www.mskcc.org/insurance-assistance/understanding-cost-your-care-msk/download"

urls = ["https://www.mskcc.org/teaser/standard-charges-nyc.json", 
        "https://www.mskcc.org/teaser/standard-charges-ny-suburbs.json",
        "https://www.mskcc.org/teaser/standard-charges-sourthern-nj.json",
        "https://www.mskcc.org/teaser/standard-charges-northern-nj.json"]
url = urls[0]
ccn = "330154"
ein = "13-1624082"
filename = derive_filename_from_url(url)
filename

'standard-charges-nyc.json'

In [4]:
subprocess.run(["wget", "--no-clobber", url, "-O", filename])

File ‘standard-charges-nyc.json’ already there; not retrieving.


CompletedProcess(args=['wget', '--no-clobber', 'https://www.mskcc.org/teaser/standard-charges-nyc.json', '-O', 'standard-charges-nyc.json'], returncode=1)

In [5]:
in_f = open(filename, "r")
json_dict = json.load(in_f)
in_f.close()
json_dict.keys()

dict_keys(['File Notes', 'OP_NYC_07272023', 'IP_NYC_07272023', 'PHY_NYC_07272023'])

In [6]:
json_dict['File Notes']

[{'Memorial Hospital for Cancer and Allied Diseases': 'New York City Outpatient, Inpatient and Physician Charges'},
 {'Memorial Hospital for Cancer and Allied Diseases': 'Updated 07/27/2023'}]

In [7]:
df_op = pd.DataFrame(json_dict['OP_NYC_07272023'])
df_op

,CHG CD,CHG CD DESC,MOD,MCD CPT,REV CD,PRICE,SELF PAY,AETNA COMMERCIAL NYC RATES,AETNA PREMIER CARE NETWORK NYC RATES,AETNA CONTRACTING METHOD,...,HEALTHFIRST MCAID & ESS 3&4 MR/DD/TBI PTS RATES,HEALTHFIRST MCAID & ESS 3&4 HOSP OP DEPT RATES,FIDELIS ESS 1&2 AMB SURG RATES,FIDELIS ESS 1&2 MR/DD/TBI PTS RATES,FIDELIS ESS 1&2 HOSP OP DEPT RATES,FIDELIS MCAID & ESS 3&4 AMB SURG RATES,FIDELIS MCAID & ESS 3&4 MR/DD/TBI PTS RATES,FIDELIS MCAID & ESS 3&4 HOSP OP DEPT RATES,MIN OP,MAX OP
0,AMB000007,HOSPITAL OUTPT VISIT - AMB,,G0463,510,$393.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AMB000010,INJECTION TENDON SHEATH/LIGMNT,,20550,490,$740.00,NaN,NaN,NaN,NaN,...,355.789637,296.510025,552.987118,622.631864,518.892544,315.992639,355.789637,296.510025,NaN,NaN
2,AMB000012,"N BLOCK INJ, CELIAC PELUS",,64530,490,"$1,604.00",NaN,NaN,NaN,NaN,...,675.262875,562.754481,1049.529366,1181.710032,984.820342,599.731066,675.262875,562.754481,NaN,NaN
3,AMB000016,SSEP STD 1 LWR LIMB - AMB,52,95926,922,$562.00,NaN,438.36,394.524,PERCENT OF CHARGE,...,166.310369,138.600697,258.488394,291.043146,242.551220,147.707654,166.310369,138.600697,NaN,NaN
4,AMB000017,SSEP STD 1 UPR LIMB- AMB,52,95925,922,$562.00,NaN,438.36,394.524,PERCENT OF CHARGE,...,166.310369,138.600697,258.488394,291.043146,242.551220,147.707654,166.310369,138.600697,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19959,SERVICE PACKAGE,NaN,NaN,S2325,NaN,NaN,NaN,NaN,NaN,CASE RATE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19960,SERVICE PACKAGE,NaN,NaN,S2340,NaN,NaN,NaN,NaN,NaN,CASE RATE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19961,SERVICE PACKAGE,NaN,NaN,S2342,NaN,NaN,NaN,NaN,NaN,CASE RATE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19962,SERVICE PACKAGE,NaN,NaN,S2411,NaN,NaN,NaN,NaN,NaN,CASE RATE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_ip = pd.DataFrame(json_dict['IP_NYC_07272023'])
df_ip

,CHG CD,CHG CD DESC,MOD,MCD CPT/MS DRG,REV CD,PRICE,SELF PAY,AETNA COMMERCIAL NYC RATES,AETNA PREMIER CARE NETWORK NYC RATES,AETNA CONTRACTING METHOD,...,URN CONTRACTING METHOD,EMPIRE HEALTHPLUS RATES,EMBLEM HIP ESSENTIAL 1&2 RATES,EMBLEM HIP MCAID & ESSENTIAL 3&4 RATES,HEALTHFIRST ESSENTIAL 1&2 RATES,HEALTHFIRST MCAID & ESSENTIAL 3&4 RATES,FIDELIS ESSENTIAL 1&2 RATES,FIDELIS MCAID & ESSENTIAL 3&4 RATES,MIN IP,MAX IP
0,AMB000007,HOSPITAL OUTPT VISIT - AMB,,G0463,510,$393.00,$393.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AMB000010,INJECTION TENDON SHEATH/LIGMNT,,20550,490,$740.00,$740.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AMB000012,"N BLOCK INJ, CELIAC PELUS",,64530,490,"$1,604.00","$1,604.00",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AMB000016,SSEP STD 1 LWR LIMB - AMB,52,95926,922,$562.00,$562.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AMB000017,SSEP STD 1 UPR LIMB- AMB,52,95925,922,$562.00,$562.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13912,SERVICE PACKAGE,Per Diem: Med/Surg/Ped/OB,NaN,NaN,Other IP,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13913,SERVICE PACKAGE,BONE MARROW TRANSPLANTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13914,SERVICE PACKAGE,CAR-T TRANSPLANTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13915,SERVICE PACKAGE,Per Diem: Surg Day 0-1,NaN,Surg DRGs,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_phy = pd.DataFrame(json_dict['PHY_NYC_07272023'])
df_phy

,CHG CD,CPT Code,Mod,Description,PRICE,SELF PAY,AETNA COMMERCIAL NYC RATES,AETNA PREMIER CARE NETWORK NYC RATES,AETNA CONTRACTING METHOD,CIGNA RATES,...,EMBLEM HIP ESS 1&2 RATES,EMBLEM MCAID & HIP ESS 3&4 RATES,HEALTHFIRST ESS 1&2 RATES,HEALTHFIRST MCAID & ESS 3&4 RATES,HEALTHFIRST MCAID & ESS CONTRACTING METHODS,FIDELIS ESS 1&2 RATES,FIDELIS MCAID & ESS 3&4 RATES,FIDELIS MCAID & ESS CONTRACTING METHODS,MIN PHY,MAX PHY
0,0100100,00100,,ANESTHESIA FOR SALIVARY GLANDS,145,145,119.3300,113.36350,Conv Factor,134.0000,...,MEDICAID RATE,MEDICAID RATE,NaN,NaN,FEE SCHEDULE,NaN,NaN,FEE SCHEDULE,NaN,NaN
1,0100102,00102,,PLASTIC REPAIR OF CLEFT LIP,145,145,119.3300,113.36350,Conv Factor,134.0000,...,MEDICAID RATE,MEDICAID RATE,NaN,NaN,FEE SCHEDULE,NaN,NaN,FEE SCHEDULE,NaN,NaN
2,0100103,00103,,BLEPHAROPLASTY,145,145,119.3300,113.36350,Conv Factor,134.0000,...,MEDICAID RATE,MEDICAID RATE,NaN,NaN,FEE SCHEDULE,NaN,NaN,FEE SCHEDULE,NaN,NaN
3,0100104,00104,,ANESTH FOR ELECTROCONVULSIVE THERAPY,145,145,119.3300,113.36350,Conv Factor,134.0000,...,MEDICAID RATE,MEDICAID RATE,NaN,NaN,FEE SCHEDULE,NaN,NaN,FEE SCHEDULE,NaN,NaN
4,0100120,00120,,ANESTHESIA FOR EAR SURGERY,145,145,119.3300,113.36350,Conv Factor,134.0000,...,MEDICAID RATE,MEDICAID RATE,NaN,NaN,FEE SCHEDULE,NaN,NaN,FEE SCHEDULE,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12987,9300452,G0452,26,MOLECULAR PATH PROC; INTRP AND RPT- TIER 1,205,205,75.7914,72.00183,FEE SCHEDULE/PERCENT OF CHARGE,31.4959,...,MEDICAID RATE,MEDICAID RATE,55.77,44.616,FEE SCHEDULE,55.77,44.616,FEE SCHEDULE,NaN,NaN
12988,9310452,G0452,26,MOLECULAR PATH PROC; INTRP AND RPT- TIER 2,205,205,75.7914,72.00183,FEE SCHEDULE/PERCENT OF CHARGE,31.4959,...,MEDICAID RATE,MEDICAID RATE,55.77,44.616,FEE SCHEDULE,55.77,44.616,FEE SCHEDULE,NaN,NaN
12989,9388387,88387,26,"MACROSCOPIC EXAM PREP,MOLEC STUDIES, EA TISS",150,150,111.6870,106.10265,FEE SCHEDULE/PERCENT OF CHARGE,183.2454,...,MEDICAID RATE,MEDICAID RATE,38.39,30.712,FEE SCHEDULE,38.39,30.712,FEE SCHEDULE,NaN,NaN
12990,J730842,J7308,,"AMINOLEVULINIC ACID HCL ADM,20%, 354MG - HAUP ...",1135,1135,567.5000,539.12500,FEE SCHEDULE/PERCENT OF CHARGE,933.0350,...,MEDICAID RATE,MEDICAID RATE,NaN,NaN,FEE SCHEDULE,NaN,NaN,FEE SCHEDULE,NaN,NaN


In [10]:
def rearrange_columns(df):
    columns = df.columns.to_list()

    new_columns = list(filter(lambda c: c.endswith("CONTRACTING METHOD"), columns))
    new_columns += list(filter(lambda c: not c.endswith("CONTRACTING METHOD"), columns)) 

    df = df.reindex(columns=new_columns)

    return df

In [11]:
setting = 'outpatient'
billing_class = 'facility'

df_mid = pd.DataFrame(df_op)

df_mid = rearrange_columns(df_mid)
df_mid = df_mid.rename(columns={
    'CHG CD': 'local_code',
    'CHG CD DESC': 'description',
    'MCD CPT': 'hcpcs_cpt',
    'MOD': 'modifiers',
    'REV CD': 'rev_code',
    'MCD CPT/MS DRG': 'code'
})

df_mid

,AETNA CONTRACTING METHOD,CIGNA CONTRACTING METHOD,EMBLEM CONTRACTING METHOD,EMPIRE CONTRACTING METHOD,UHC CONTRACTING METHOD,HEALTHFIRST CONTRACTING METHOD,MAGNACARE CONTRACTING METHOD,METROPLUS CONTRACTING METHOD,MULTIPLAN CONTRACTING METHOD,URN CONTRACTING METHOD,...,HEALTHFIRST MCAID & ESS 3&4 MR/DD/TBI PTS RATES,HEALTHFIRST MCAID & ESS 3&4 HOSP OP DEPT RATES,FIDELIS ESS 1&2 AMB SURG RATES,FIDELIS ESS 1&2 MR/DD/TBI PTS RATES,FIDELIS ESS 1&2 HOSP OP DEPT RATES,FIDELIS MCAID & ESS 3&4 AMB SURG RATES,FIDELIS MCAID & ESS 3&4 MR/DD/TBI PTS RATES,FIDELIS MCAID & ESS 3&4 HOSP OP DEPT RATES,MIN OP,MAX OP
0,NaN,FLAT,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,355.789637,296.510025,552.987118,622.631864,518.892544,315.992639,355.789637,296.510025,NaN,NaN
2,NaN,NaN,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,675.262875,562.754481,1049.529366,1181.710032,984.820342,599.731066,675.262875,562.754481,NaN,NaN
3,PERCENT OF CHARGE,FLAT,PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,166.310369,138.600697,258.488394,291.043146,242.551220,147.707654,166.310369,138.600697,NaN,NaN
4,PERCENT OF CHARGE,FLAT,PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,166.310369,138.600697,258.488394,291.043146,242.551220,147.707654,166.310369,138.600697,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19959,CASE RATE,CASE RATE,NaN,CASE RATE,CASE RATE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19960,CASE RATE,CASE RATE,NaN,CASE RATE,CASE RATE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19961,CASE RATE,CASE RATE,NaN,CASE RATE,CASE RATE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19962,CASE RATE,CASE RATE,NaN,CASE RATE,CASE RATE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
money_idx = df_mid.columns.to_list().index("PRICE")
money_columns = df_mid.columns.to_list()[money_idx:]
money_columns

['PRICE',
 'SELF PAY',
 'AETNA COMMERCIAL NYC RATES',
 'AETNA PREMIER CARE NETWORK NYC RATES',
 'CIGNA RATES',
 'EMBLEM GHI RATES',
 'EMBLEM CBP RATES',
 'EMBLEM HIP RATES',
 'EMBLEM SELECT CARE RATES',
 'EMPIRE COMMERCIAL RATES',
 'EMPIRE BLUE ACCESS RATES',
 'EMPIRE CONNECTIONS RATES',
 'NY UHC COMMERCIAL RATES',
 'NY OXFORD FREEDOM RATES',
 'NY OXFORD LIBERTY RATES',
 'NY OXFORD METRO RATES',
 'NY UHC CORE RATES',
 'NY UHC CHARTER RATES',
 'HEALTHFIRST RATES',
 'HEALTHFIRST PERCENT',
 'MAGNACARE RATES',
 'MAGNACARE JIB RATES',
 'METROPLUS SMALL BUSINESS RATES',
 'METROPLUS SMALL BUSINESS PERCENT',
 'METROPLUS CITY EMPLOYEE HEALTH RATES',
 'METROPLUS CITY EMPLOYEE HEALTH PERCENT',
 'MULTIPLAN RATES',
 'URN RATES',
 'EMPIRE HEALTHPLUS AMB SURG RATES',
 'EMPIRE HEALTHPLUS MR/DD/TBI PTS RATES',
 'EMPIRE HEALTHPLUS HOSP OP DEPT RATES',
 'EMBLEM HIP ESS 1&2 AMB SURG RATES',
 'EMBLEM HIP ESS 1&2 MR/DD/TBI PTS RATES',
 'EMBLEM HIP ESS 1&2 HOSP OP DEPT RATES',
 'EMBLEM MCAID & HIP ESS 3&4 AM

In [13]:
remaining_columns = df_mid.columns.to_list()[:money_idx]
df_mid = pd.melt(df_mid, id_vars=remaining_columns, var_name='payer_name', value_name='standard_charge')
df_mid

,AETNA CONTRACTING METHOD,CIGNA CONTRACTING METHOD,EMBLEM CONTRACTING METHOD,EMPIRE CONTRACTING METHOD,UHC CONTRACTING METHOD,HEALTHFIRST CONTRACTING METHOD,MAGNACARE CONTRACTING METHOD,METROPLUS CONTRACTING METHOD,MULTIPLAN CONTRACTING METHOD,URN CONTRACTING METHOD,local_code,description,modifiers,hcpcs_cpt,rev_code,payer_name,standard_charge
0,NaN,FLAT,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,AMB000007,HOSPITAL OUTPT VISIT - AMB,,G0463,510,PRICE,$393.00
1,NaN,NaN,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,AMB000010,INJECTION TENDON SHEATH/LIGMNT,,20550,490,PRICE,$740.00
2,NaN,NaN,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,AMB000012,"N BLOCK INJ, CELIAC PELUS",,64530,490,PRICE,"$1,604.00"
3,PERCENT OF CHARGE,FLAT,PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,AMB000016,SSEP STD 1 LWR LIMB - AMB,52,95926,922,PRICE,$562.00
4,PERCENT OF CHARGE,FLAT,PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,AMB000017,SSEP STD 1 UPR LIMB- AMB,52,95925,922,PRICE,$562.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018159,CASE RATE,CASE RATE,NaN,CASE RATE,CASE RATE,NaN,NaN,NaN,NaN,NaN,SERVICE PACKAGE,NaN,NaN,S2325,NaN,MAX OP,NaN
1018160,CASE RATE,CASE RATE,NaN,CASE RATE,CASE RATE,NaN,NaN,NaN,NaN,NaN,SERVICE PACKAGE,NaN,NaN,S2340,NaN,MAX OP,NaN
1018161,CASE RATE,CASE RATE,NaN,CASE RATE,CASE RATE,NaN,NaN,NaN,NaN,NaN,SERVICE PACKAGE,NaN,NaN,S2342,NaN,MAX OP,NaN
1018162,CASE RATE,CASE RATE,NaN,CASE RATE,CASE RATE,NaN,NaN,NaN,NaN,NaN,SERVICE PACKAGE,NaN,NaN,S2411,NaN,MAX OP,NaN


In [14]:
df_mid['standard_charge_percent'] = None

df_mid.loc[df_mid['payer_name'].str.endswith('PERCENT'), 'standard_charge_percent'] = df_mid[df_mid['payer_name'].str.endswith('PERCENT')]['standard_charge']
df_mid.loc[df_mid['payer_name'].str.endswith('PERCENT'), 'standard_charge'] = None

set(df_mid['standard_charge_percent'].to_list())

{'66%', '67%', None, nan}

In [15]:
set(df_mid['modifiers'].to_list())

{' ',
 '50 ',
 '52 ',
 '52 LT',
 '52 RT',
 '91 ',
 '95 ',
 '95 PO',
 'CS ',
 'GN ',
 'GN 95',
 'GO ',
 'GO 95',
 'GP ',
 'GP 95',
 'GY ',
 'JA ',
 'JB ',
 'JW ',
 'LT ',
 'Q0 ',
 'RT ',
 nan}

In [16]:
def fix_modifiers(modifiers):
    modifiers = modifiers.strip()
    if modifiers == '':
        return None

    return modifiers.replace(" ", "|")

df_mid.loc[df_mid['modifiers'].notnull(), 'modifiers'] = df_mid[df_mid['modifiers'].notnull()]['modifiers'].apply(fix_modifiers)
set(df_mid['modifiers'].to_list())

{'50',
 '52',
 '52|LT',
 '52|RT',
 '91',
 '95',
 '95|PO',
 'CS',
 'GN',
 'GN|95',
 'GO',
 'GO|95',
 'GP',
 'GP|95',
 'GY',
 'JA',
 'JB',
 'JW',
 'LT',
 None,
 'Q0',
 'RT',
 nan}

In [17]:
def payer_name_to_rate_category(payer_name):
    if payer_name.startswith("MAX"):
        return 'max'
    elif payer_name.startswith("MIN"):
        return 'min'
    elif payer_name == "PRICE":
        return 'gross'
    elif payer_name == "SELF PAY":
        return 'cash'

    return 'negotiated'

df_mid['rate_category'] = df_mid['payer_name'].apply(payer_name_to_rate_category)
df_mid[['payer_name', 'rate_category']].value_counts()

payer_name                                       rate_category
AETNA COMMERCIAL NYC RATES                       negotiated       19964
AETNA PREMIER CARE NETWORK NYC RATES             negotiated       19964
CIGNA RATES                                      negotiated       19964
EMBLEM CBP RATES                                 negotiated       19964
EMBLEM GHI RATES                                 negotiated       19964
EMBLEM HIP ESS 1&2 AMB SURG RATES                negotiated       19964
EMBLEM HIP ESS 1&2 HOSP OP DEPT RATES            negotiated       19964
EMBLEM HIP ESS 1&2 MR/DD/TBI PTS RATES           negotiated       19964
EMBLEM HIP RATES                                 negotiated       19964
EMBLEM MCAID & HIP ESS 3&4 AMB SURG RATES        negotiated       19964
EMBLEM MCAID & HIP ESS 3&4 HOSP OP DEPT RATES    negotiated       19964
EMBLEM MCAID & HIP ESS 3&4 MR/DD/TBI PTS RATES   negotiated       19964
EMBLEM SELECT CARE RATES                         negotiated       19964
E

In [18]:
df_mid['contracting_method'] = None
df_mid['additional_payer_specific_notes'] = None

def set_contracting_method(row):
    if row.get('rate_category') != "negotiated":
        return row
    
    method_by_payer = dict()
    
    for col_name in dict(row).keys():
        value = row[col_name]
        if type(value) != str:
            continue
        
        if 'CONTRACTING METHOD' in col_name:
            payer = col_name.replace("CONTRACTING METHOD", "")
            method_by_payer[payer] = value
    
    payer_name = row.get('payer_name')

    for payer in method_by_payer.keys():
        if not payer in payer_name:
            continue

        method = method_by_payer[payer]
        if method == 'FEE SCHEDULE':
            row['contracting_method'] = 'fee schedule'
        elif method == 'PERCENT OF CHARGE':
            row['contracting_method'] = 'percent of total billed charge'
        elif method == 'CASE RATE':
            row['contracting_method'] = 'case rate'
        elif method == 'PER DIEM':
            row['contracting_method'] = 'per diem'
        else:
            row['contracting_method'] = 'other'
            row['additional_payer_specific_notes'] = method

    return row

df_mid = df_mid.apply(set_contracting_method, axis=1)
df_mid[df_mid['rate_category'] == 'negotiated']

,AETNA CONTRACTING METHOD,CIGNA CONTRACTING METHOD,EMBLEM CONTRACTING METHOD,EMPIRE CONTRACTING METHOD,UHC CONTRACTING METHOD,HEALTHFIRST CONTRACTING METHOD,MAGNACARE CONTRACTING METHOD,METROPLUS CONTRACTING METHOD,MULTIPLAN CONTRACTING METHOD,URN CONTRACTING METHOD,...,description,modifiers,hcpcs_cpt,rev_code,payer_name,standard_charge,standard_charge_percent,rate_category,contracting_method,additional_payer_specific_notes
39928,NaN,FLAT,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,HOSPITAL OUTPT VISIT - AMB,None,G0463,510,AETNA COMMERCIAL NYC RATES,NaN,None,negotiated,None,None
39929,NaN,NaN,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,INJECTION TENDON SHEATH/LIGMNT,None,20550,490,AETNA COMMERCIAL NYC RATES,NaN,None,negotiated,None,None
39930,NaN,NaN,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,"N BLOCK INJ, CELIAC PELUS",None,64530,490,AETNA COMMERCIAL NYC RATES,NaN,None,negotiated,None,None
39931,PERCENT OF CHARGE,FLAT,PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,SSEP STD 1 LWR LIMB - AMB,52,95926,922,AETNA COMMERCIAL NYC RATES,438.36,None,negotiated,percent of total billed charge,None
39932,PERCENT OF CHARGE,FLAT,PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,SSEP STD 1 UPR LIMB- AMB,52,95925,922,AETNA COMMERCIAL NYC RATES,438.36,None,negotiated,percent of total billed charge,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978231,CASE RATE,CASE RATE,NaN,CASE RATE,CASE RATE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,S2325,NaN,FIDELIS MCAID & ESS 3&4 HOSP OP DEPT RATES,NaN,None,negotiated,None,None
978232,CASE RATE,CASE RATE,NaN,CASE RATE,CASE RATE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,S2340,NaN,FIDELIS MCAID & ESS 3&4 HOSP OP DEPT RATES,NaN,None,negotiated,None,None
978233,CASE RATE,CASE RATE,NaN,CASE RATE,CASE RATE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,S2342,NaN,FIDELIS MCAID & ESS 3&4 HOSP OP DEPT RATES,NaN,None,negotiated,None,None
978234,CASE RATE,CASE RATE,NaN,CASE RATE,CASE RATE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,S2411,NaN,FIDELIS MCAID & ESS 3&4 HOSP OP DEPT RATES,NaN,None,negotiated,None,None


In [19]:
df_mid = df_mid[df_mid['standard_charge'].notnull() | df_mid['standard_charge_percent'].notnull()]
df_mid

,AETNA CONTRACTING METHOD,CIGNA CONTRACTING METHOD,EMBLEM CONTRACTING METHOD,EMPIRE CONTRACTING METHOD,UHC CONTRACTING METHOD,HEALTHFIRST CONTRACTING METHOD,MAGNACARE CONTRACTING METHOD,METROPLUS CONTRACTING METHOD,MULTIPLAN CONTRACTING METHOD,URN CONTRACTING METHOD,...,description,modifiers,hcpcs_cpt,rev_code,payer_name,standard_charge,standard_charge_percent,rate_category,contracting_method,additional_payer_specific_notes
0,NaN,FLAT,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,HOSPITAL OUTPT VISIT - AMB,None,G0463,510,PRICE,$393.00,None,gross,None,None
1,NaN,NaN,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,INJECTION TENDON SHEATH/LIGMNT,None,20550,490,PRICE,$740.00,None,gross,None,None
2,NaN,NaN,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,"N BLOCK INJ, CELIAC PELUS",None,64530,490,PRICE,"$1,604.00",None,gross,None,None
3,PERCENT OF CHARGE,FLAT,PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,SSEP STD 1 LWR LIMB - AMB,52,95926,922,PRICE,$562.00,None,gross,None,None
4,PERCENT OF CHARGE,FLAT,PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,SSEP STD 1 UPR LIMB- AMB,52,95925,922,PRICE,$562.00,None,gross,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971406,NaN,NaN,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,CLOSED TREATMNT-HUMERAL SHAFT FRACT- W/O MALIPLTN,None,24500,456,FIDELIS MCAID & ESS 3&4 HOSP OP DEPT RATES,508.536843,None,negotiated,None,None
971407,NaN,NaN,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,"SIGMOIDOSCOPY- FLEX, W/ BLEEDNG CNTRL-ANY METH",None,45334,456,FIDELIS MCAID & ESS 3&4 HOSP OP DEPT RATES,782.715336,None,negotiated,None,None
971408,NaN,NaN,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,CLOSED TRTMNT- DISTAL RADIAL FRACTR,None,25605,456,FIDELIS MCAID & ESS 3&4 HOSP OP DEPT RATES,508.536843,None,negotiated,None,None
971409,NaN,NaN,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,"DRESSNG/DEBRIBMNT OR PARTL-THCKNSS BURNS,INTL ...",None,16020,456,FIDELIS MCAID & ESS 3&4 HOSP OP DEPT RATES,463.332315,None,negotiated,None,None


In [20]:
df_mid['standard_charge'] = df_mid['standard_charge'].apply(cleanup_dollar_value)
df_mid['standard_charge']

0             393.00
1             740.00
2            1604.00
3             562.00
4             562.00
             ...    
971406    508.536843
971407    782.715336
971408    508.536843
971409    463.332315
971410    135.468396
Name: standard_charge, Length: 451473, dtype: object

In [21]:
df_mid['rev_code'] = df_mid['rev_code'].apply(pad_rev_code_if_needed)
df_mid['rev_code']

0         0510
1         0490
2         0490
3         0922
4         0922
          ... 
971406    0456
971407    0456
971408    0456
971409    0456
971410    0456
Name: rev_code, Length: 451473, dtype: object

In [22]:
df_mid['hospital_id'] = ccn
df_mid['line_type'] = None
if not 'code' in df_mid.columns:
    df_mid['code'] = None
if not 'ms_drg' in df_mid.columns:
    df_mid['ms_drg'] = None
df_mid['apr_drg'] = None
df_mid['eapg'] = None
df_mid['alt_hcpcs_cpt'] = None
df_mid['thru'] = None
df_mid['apc'] = None
df_mid['icd'] = None
df_mid['ndc'] = None
df_mid['drug_hcpcs_multiplier'] = None
df_mid['drug_quantity'] = None
df_mid['drug_unit_of_measurement'] = None
df_mid['drug_type_of_measurement'] = None
df_mid['billing_class'] = None
df_mid['setting'] = None
df_mid['plan_name'] = None
df_mid['additional_generic_notes'] = None
if not 'additional_payer_specific_notes' in df_mid.columns:
    df_mid['additional_payer_specific_notes'] = None

df_mid

,AETNA CONTRACTING METHOD,CIGNA CONTRACTING METHOD,EMBLEM CONTRACTING METHOD,EMPIRE CONTRACTING METHOD,UHC CONTRACTING METHOD,HEALTHFIRST CONTRACTING METHOD,MAGNACARE CONTRACTING METHOD,METROPLUS CONTRACTING METHOD,MULTIPLAN CONTRACTING METHOD,URN CONTRACTING METHOD,...,icd,ndc,drug_hcpcs_multiplier,drug_quantity,drug_unit_of_measurement,drug_type_of_measurement,billing_class,setting,plan_name,additional_generic_notes
0,NaN,FLAT,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,None,None,None,None,None,None,None,None,None,None
1,NaN,NaN,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,None,None,None,None,None,None,None,None,None,None
2,NaN,NaN,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,None,None,None,None,None,None,None,None,None,None
3,PERCENT OF CHARGE,FLAT,PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,None,None,None,None,None,None,None,None,None,None
4,PERCENT OF CHARGE,FLAT,PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971406,NaN,NaN,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,None,None,None,None,None,None,None,None,None,None
971407,NaN,NaN,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,None,None,None,None,None,None,None,None,None,None
971408,NaN,NaN,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,None,None,None,None,None,None,None,None,None,None
971409,NaN,NaN,PERCENT OF CHARGE,NaN,NaN,FEE SCHEDULE/PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGE,PERCENT OF CHARGES,...,None,None,None,None,None,None,None,None,None,None


In [23]:
df_out = pd.DataFrame(df_mid[TARGET_COLUMNS])
df_out

,hospital_id,line_type,description,rev_code,local_code,code,ms_drg,apr_drg,eapg,hcpcs_cpt,...,billing_class,setting,rate_category,payer_name,plan_name,standard_charge,standard_charge_percent,contracting_method,additional_generic_notes,additional_payer_specific_notes
0,330154,None,HOSPITAL OUTPT VISIT - AMB,0510,AMB000007,None,None,None,None,G0463,...,None,None,gross,PRICE,None,393.00,None,None,None,None
1,330154,None,INJECTION TENDON SHEATH/LIGMNT,0490,AMB000010,None,None,None,None,20550,...,None,None,gross,PRICE,None,740.00,None,None,None,None
2,330154,None,"N BLOCK INJ, CELIAC PELUS",0490,AMB000012,None,None,None,None,64530,...,None,None,gross,PRICE,None,1604.00,None,None,None,None
3,330154,None,SSEP STD 1 LWR LIMB - AMB,0922,AMB000016,None,None,None,None,95926,...,None,None,gross,PRICE,None,562.00,None,None,None,None
4,330154,None,SSEP STD 1 UPR LIMB- AMB,0922,AMB000017,None,None,None,None,95925,...,None,None,gross,PRICE,None,562.00,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971406,330154,None,CLOSED TREATMNT-HUMERAL SHAFT FRACT- W/O MALIPLTN,0456,UCC000130,None,None,None,None,24500,...,None,None,negotiated,FIDELIS MCAID & ESS 3&4 HOSP OP DEPT RATES,None,508.536843,None,None,None,None
971407,330154,None,"SIGMOIDOSCOPY- FLEX, W/ BLEEDNG CNTRL-ANY METH",0456,UCC000131,None,None,None,None,45334,...,None,None,negotiated,FIDELIS MCAID & ESS 3&4 HOSP OP DEPT RATES,None,782.715336,None,None,None,None
971408,330154,None,CLOSED TRTMNT- DISTAL RADIAL FRACTR,0456,UCC000132,None,None,None,None,25605,...,None,None,negotiated,FIDELIS MCAID & ESS 3&4 HOSP OP DEPT RATES,None,508.536843,None,None,None,None
971409,330154,None,"DRESSNG/DEBRIBMNT OR PARTL-THCKNSS BURNS,INTL ...",0456,UCC000133,None,None,None,None,16020,...,None,None,negotiated,FIDELIS MCAID & ESS 3&4 HOSP OP DEPT RATES,None,463.332315,None,None,None,None


In [ ]:
df_out.to_csv